In [213]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import MaxPooling2D, Dense, Flatten, Conv2D
from keras.callbacks import EarlyStopping
from keras import backend as K
import glob

In [160]:
img_height=80
img_width=80
channel=3
X_train=np.empty([8005, img_height, img_width, channel], np.int32)
X_test=np.empty([2023, img_height, img_width, channel], np.int32)
y_train=np.full([8005,], fill_value=5)
y_test=np.full([2023,], fill_value=6)
i = 0
# cat = 0 , dog =1

In [161]:
for filepath in glob.iglob('training_set/cats/*.jpg'):
    img=Image.open(filepath)
    img=img.resize((img_height,img_width), Image.ANTIALIAS)
    img=np.array(img)
    y_train[i] = 0
    X_train[i] = img
    i = i + 1
    
for filepath in glob.iglob('training_set/dogs/*.jpg'):
    img=Image.open(filepath)
    img=img.resize((img_height,img_width), Image.ANTIALIAS)
    img=np.array(img)
    y_train[i] = 1
    X_train[i] = img
    i = i + 1    
    
i=0    
    
for filepath in glob.iglob('test_set/cats/*.jpg'):
    img=Image.open(filepath)
    img=img.resize((img_height,img_width), Image.ANTIALIAS)
    img=np.array(img)
    y_test[i] = 0
    X_test[i] = img
    i = i + 1
    
for filepath in glob.iglob('test_set/dogs/*.jpg'):
    img=Image.open(filepath)
    img=img.resize((img_height,img_width), Image.ANTIALIAS)
    img=np.array(img)
    y_test[i] = 1
    X_test[i] = img
    i = i + 1      

In [162]:
X_train = X_train/255
X_test  = X_test/255
y_train = keras.utils.to_categorical(y_train, 2)
y_test  = keras.utils.to_categorical(y_test, 2)
input_shape=(img_height, img_width, channel)

In [208]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(2,2), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [214]:
early_stopping_monitor = EarlyStopping(patience=2)
model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(X_test, y_test), callbacks=[early_stopping_monitor])
score=model.evaluate(X_test, y_test)
print('Loss:', score[0])
print('Accuracy:',score[1])

Train on 8005 samples, validate on 2023 samples
Epoch 1/10
8005/8005 [==============================] - 28s 4ms/step - loss: 0.3914 - accuracy: 0.8216 - val_loss: 0.4703 - val_accuracy: 0.7855
Epoch 2/10
8005/8005 [==============================] - 28s 4ms/step - loss: 0.3836 - accuracy: 0.8234 - val_loss: 0.4651 - val_accuracy: 0.7919
Epoch 3/10
8005/8005 [==============================] - 29s 4ms/step - loss: 0.3560 - accuracy: 0.8446 - val_loss: 0.4623 - val_accuracy: 0.7949
Epoch 4/10
8005/8005 [==============================] - 31s 4ms/step - loss: 0.3396 - accuracy: 0.8512 - val_loss: 0.4672 - val_accuracy: 0.7909
Epoch 5/10
2023/2023 [==============================] - 2s 964us/step
Loss: 0.48554118811436003
Accuracy: 0.7810183167457581


In [211]:
model.save('cat_dog.h5')

This model achieved an accuracy of 78%. Data augmentation can probably be used to improve accuracy.